<a href="https://colab.research.google.com/github/jathusharini/intrusion-detection-fewshot-vs-traditional/blob/main/traditional_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#importing necessary libraries that will be used throughout the project
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

In [28]:
#Load the Subset Dataset
file_path = "cleaned_dataset.csv"
df = pd.read_csv(file_path)

In [29]:
# Encoding Categorical Features
label_encoder = LabelEncoder()
if 'Label' in df.columns:
    df['Label'] = label_encoder.fit_transform(df['Label'])

In [30]:
# Data Splitting
X = df.drop(['Attack Type', 'Label'], axis=1, errors='ignore')
y = df['Label']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


In [31]:
#Model Training and Evaluation
models = [
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("AdaBoost Classifier", AdaBoostClassifier()),
    ("Knn Classifier", KNeighborsClassifier())
]

scores = []
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = f1_score(y_val, y_pred, average='weighted')
    scores.append((name, score))

print("Model Performance on Validation Set:")
for name, score in scores:
    print(f'{name} F1 score: {score}')

In [31]:
# Hyperparameter Tuning for Random Forest
model = RandomForestClassifier(random_state=42)
n_estimators = [5, 10, 15]
max_depths = [5, 10, 15]
tuning_results = []
for n in n_estimators:
    for d in max_depths:
        model.set_params(n_estimators=n, max_depth=d)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        score = f1_score(y_val, y_pred, average='weighted')
        tuning_results.append((n, d, score))

print("\nHyperparameter Tuning Results:")
for n, d, score in tuning_results:
    print(f'n_estimators={n}, max_depth={d} F1 score: {score}')